# LEAP3D Interface Demo

## Config

In [ ]:
from leap3d.config import DATA_DIR, DATASET_DIR, X_MIN, X_MAX, Y_MIN, Y_MAX, Z_MIN, Z_MAX, SAFETY_OFFSET, MELTING_POINT, TIMESTEP_DURATION, SCALE_DISTANCE_BY, SCALE_TIME_BY, BASE_TEMPERATURE

## ScanParameters

In [ ]:
from leap3d.scanning import ScanParameters

In [ ]:
case_index = 0
params_filename =  DATA_DIR / "Params.npy"
rough_coord_filename =  DATA_DIR / "Rough_coord.npz"

In [ ]:
case_params = ScanParameters(params_filename, rough_coord_filename, case_index,
                             x_min=X_MIN, x_max=X_MAX,
                             y_min=Y_MIN, y_max=Y_MAX,
                             z_min=Z_MIN, z_max=Z_MAX,
                             safety_offset=SAFETY_OFFSET,
                             melting_point=MELTING_POINT,
                             timestep_duration=TIMESTEP_DURATION,
                             scale_distance_by=SCALE_DISTANCE_BY,
                             scale_time_by=SCALE_TIME_BY)

Object bounds

In [ ]:
x_min, x_max, y_min, y_max, z_min, z_max = case_params.get_bounds()
x_min, x_max, y_min, y_max, z_min, z_max

Scanning bounds

In [ ]:
laser_x_min, laser_x_max, laser_y_min, laser_y_max = case_params.get_laser_bounds()
laser_x_min, laser_x_max, laser_y_min, laser_y_max

Edge points of current laser trajectory

In [ ]:
laser_x_min, laser_x_max, laser_y_min, laser_y_max = case_params.get_cross_section_scanning_bounds(laser_x=0, laser_y=0)
laser_x_min, laser_x_max, laser_y_min, laser_y_max

## ScanResults

In [ ]:
from leap3d.scanning import ScanResults

In [ ]:
case_index = 0
case_filename = DATA_DIR / "case_0000.npz"

In [ ]:
case_results = ScanResults(case_filename)

In [ ]:
type(case_results.get_laser_data_at_timestep(0)[0])


In [ ]:
case_results.get_rough_temperatures_at_timestep(timestep=0).shape

In [ ]:
top_layer_temperatures = case_results.get_top_layer_temperatures(timestep=0)
top_layer_temperatures.shape

In [ ]:
laser_x, laser_y = case_results.get_laser_coordinates_at_timestep(timestep=0)
laser_x, laser_y

In [ ]:
coordinates, temperatures = case_results.get_coordinate_points_and_temperature_at_timestep(case_params, timestep=0)
coordinates, temperatures

## Dataset

### Dataset preparation

In [ ]:
from leap3d import dataset as dataset
from leap3d.dataset import check_if_prepared_data_exists

In [ ]:
if not check_if_prepared_data_exists(DATASET_DIR / "dataset.hdf5"):
    dataset.prepare_raw_data(params_filename, rough_coord_filename, DATA_DIR, DATASET_DIR / "dataset.hdf5", cases=8)

In [ ]:
if not check_if_prepared_data_exists(DATASET_DIR / "test_dataset.hdf5"):
    dataset.prepare_raw_data(params_filename, rough_coord_filename, DATA_DIR, DATASET_DIR / "test_dataset.hdf5", cases=[9,10])

### Dataset

In [ ]:
from leap3d.dataset import LEAP3DDataset

In [ ]:
dataset = LEAP3DDataset(DATASET_DIR, train=True)

In [ ]:
len(dataset)

In [ ]:
temperature_mean, temperature_variance, target_mean, target_variance = dataset.get_temperature_mean_and_variance(print_progress=True)

In [ ]:
print(temperature_mean, temperature_variance, target_mean, target_variance)

In [ ]:
from torchvision.transforms import transforms

from leap3d.transforms import normalize_temperature_2d


normalize_temperature = lambda x: normalize_temperature_2d(x, melting_point=MELTING_POINT, base_temperature=BASE_TEMPERATURE, inplace=True)

train_transforms = transforms.Compose([
        transforms.Lambda(normalize_temperature)
    ])

target_transforms = transforms.Compose([
    transforms.Lambda(normalize_temperature)
])
dataset = LEAP3DDataset(DATASET_DIR, train=True, transform=train_transforms, target_transform=target_transforms)

In [ ]:
normalized_temperature_mean, normalized_temperature_variance, normalized_target_mean, normalized_target_variance = dataset.get_temperature_mean_and_variance(print_progress=True)

In [ ]:
print(normalized_temperature_mean, normalized_temperature_variance, normalized_target_mean, normalized_target_variance)

In [ ]:
normalize_temperature = lambda x: normalize_temperature_2d(x, melting_point=MELTING_POINT, base_temperature=0, inplace=True)

train_transforms = transforms.Compose([
        transforms.Lambda(normalize_temperature)
    ])

target_transforms = transforms.Compose([
    transforms.Lambda(normalize_temperature)
])
dataset = LEAP3DDataset(DATASET_DIR, train=True, transform=train_transforms, target_transform=target_transforms)

In [ ]:
normalized_temperature_mean, normalized_temperature_variance, normalized_target_mean, normalized_target_variance = dataset.get_temperature_mean_and_variance(print_progress=True)

In [ ]:
print(normalized_temperature_mean, normalized_temperature_variance, normalized_target_mean, normalized_target_variance)

In [ ]:
x_train, extra_params, target = dataset[0]
x_train.shape, extra_params.shape, target.shape